In [1]:
import sys
import pandas as pd
import numpy as np
from app_utils import AppUtils
from app_utils.app_alerts import AlertService
from datetime import datetime, timedelta

In [2]:
# TESTING INITIALIZATION AND CONFIGURATION

# Create a custom configuration for testing
test_config = {
    "percentile_categories": {
        "SB": 5.0,  # Customize thresholds for testing
        "B": 20.0,
        "N": 80.0,
        "G": 95.0,
        "SG": 100
    },
    "feature_config": {
        "test_feature": {
            "description": "Test Feature",
            "importance": "high"
        }
    }
}

In [3]:
alert_service = AlertService(config=test_config)

# Test percentile mapping
test_percentiles = [1.0, 10.0, 50.0, 85.0, 98.0]
for p in test_percentiles:
    category = alert_service.map_percentile_to_category(p)
    description = alert_service.get_category_description(category)
    print(f"Percentile {p:.1f} -> Category: {category} ({description})")

# Verify configuration was applied correctly
print("\nConfiguration:")
print(f"Percentile Categories: {alert_service.config['percentile_categories']}")
print(f"Feature Config: {alert_service.config['feature_config']}")

# Verify AppUtils integration
utils = AppUtils()
alert_service.set_app_utils(utils)
print(f"\nAppUtils set: {alert_service.app_utils is not None}")

# Test validation of analyzers (should be False since we haven't initialized them yet)
print(f"Analyzers validated: {alert_service._validate_analyzers()}")

Percentile 1.0 -> Category: SB (Significantly Below Average)
Percentile 10.0 -> Category: B (Below Average)
Percentile 50.0 -> Category: N (Average)
Percentile 85.0 -> Category: G (Above Average)
Percentile 98.0 -> Category: SG (Significantly Above Average)

Configuration:
Percentile Categories: {'SB': 5.0, 'B': 20.0, 'N': 80.0, 'G': 95.0, 'SG': 100}
Feature Config: {'test_feature': {'description': 'Test Feature', 'importance': 'high'}}

AppUtils set: True
Analyzers validated: False


In [4]:
# Create sample data for testing
def create_sample_session_data():
    """Create sample session data for testing"""
    # Create dates for the last 10 days
    dates = [datetime.now() - timedelta(days=i) for i in range(10)]
    
    # Create sample data for 3 subjects
    subjects = ['sub001', 'sub002', 'sub003']
    
    # Create dataframe rows
    rows = []
    for subject in subjects:
        for i, date in enumerate(dates):
            # Sample feature values
            trials = 100 - i * 5  # Decreasing trials (100, 95, 90, ...)
            errors = i * 2        # Increasing errors (0, 2, 4, ...)
            
            # Add some variability for subject 2
            if subject == 'sub002':
                trials = trials * 0.8  # 20% fewer trials
                errors = errors * 1.5  # 50% more errors
            
            # Add each session
            rows.append({
                'subject_id': subject,
                'session_date': date,
                'session': f"session_{i+1}",
                'total_trials': trials,
                'error_count': errors
            })
    
    # Create dataframe
    return pd.DataFrame(rows)

In [5]:

# Initialize AppUtils and AlertService
utils = AppUtils()
alert_service = AlertService(app_utils=utils)

# Create sample data
sample_data = create_sample_session_data()
print(f"Created sample data with {len(sample_data)} sessions for {len(sample_data['subject_id'].unique())} subjects")

# Create feature thresholds for testing
feature_thresholds = {
    'total_trials': {
        'lower': 80  # Alert if trials are below 80
    },
    'error_count': {
        'upper': 10  # Alert if errors are above 10
    }
}

Created sample data with 30 sessions for 3 subjects


In [6]:
# Skip the real initialization and mock directly
print("Setting up mock analyzers...")

class MockThresholdAnalyzer:
    def __init__(self, sample_data, thresholds):
        self.sample_data = sample_data
        self.thresholds = thresholds
        
        # Add threshold crossing columns
        self.threshold_data = sample_data.copy()
        for feature, config in thresholds.items():
            if 'lower' in config:
                self.threshold_data[f"{feature}_above_lower"] = self.threshold_data[feature] >= config['lower']
            if 'upper' in config:
                self.threshold_data[f"{feature}_below_upper"] = self.threshold_data[feature] <= config['upper']
        
        # Create summary data
        summary_data = []
        for subject in sample_data['subject_id'].unique():
            subject_df = self.threshold_data[self.threshold_data['subject_id'] == subject]
            summary = {'subject_id': subject, 'total_sessions': len(subject_df)}
            
            # Process each threshold
            for feature, config in thresholds.items():
                if 'lower' in config:
                    col = f"{feature}_above_lower"
                    passed = subject_df[col].sum()
                    summary[f"{col}_count"] = passed
                    summary[f"{col}_percent"] = (passed / len(subject_df)) * 100
                    if passed > 0:
                        summary[f"{col}_first_date"] = subject_df[subject_df[col]]['session_date'].min()
                
                if 'upper' in config:
                    col = f"{feature}_below_upper"
                    passed = subject_df[col].sum()
                    summary[f"{col}_count"] = passed
                    summary[f"{col}_percent"] = (passed / len(subject_df)) * 100
                    if passed > 0:
                        summary[f"{col}_first_date"] = subject_df[subject_df[col]]['session_date'].min()
            
            summary_data.append(summary)
        
        self.summary_data = pd.DataFrame(summary_data)

class MockQuantileAnalyzer:
    """Empty mock for the quantile analyzer"""
    def __init__(self):
        pass

# Create mock analyzers and attach to utils
mock_threshold = MockThresholdAnalyzer(sample_data, feature_thresholds)
mock_quantile = MockQuantileAnalyzer()

Setting up mock analyzers...


In [7]:
utils.threshold_analyzer = mock_threshold
utils.quantile_analyzer = mock_quantile

# Add mock methods with the EXACT method names from your alert_service.py
utils.get_threshold_crossing = lambda subject_ids=None, start_date=None, end_date=None: mock_threshold.threshold_data.copy()
utils.get_subject_threshold_summary = lambda subject_ids=None: mock_threshold.summary_data.copy()

# Override the validation method for testing
original_validate = alert_service._validate_analyzers
alert_service._validate_analyzers = lambda: True

In [8]:
# Now try calculating alerts
print("Calculating threshold alerts...")
alerts = alert_service.calculate_threshold_alerts()
print(f"Generated alerts for {len(alerts)} subjects")

# Display alerts for each subject
for subject_id, feature_alerts in alerts.items():
    print(f"\nSubject: {subject_id}")
    for feature, conditions in feature_alerts.items():
        print(f"  Feature: {feature}")
        for condition, details in conditions.items():
            value_str = f"value={details['value']}" if details['value'] is not None else ""
            count_str = f"count={details['crossing_count']}" if details['crossing_count'] is not None else ""
            print(f"    {condition}: {value_str} {count_str}")

# Test getting subjects with alerts
subjects_with_alerts = alert_service.get_subjects_with_threshold_alerts()
print(f"\nSubjects with any alert: {subjects_with_alerts}")

# Test getting subjects with specific feature alerts
subjects_with_trial_alerts = alert_service.get_subjects_with_threshold_alerts(['total_trials'])
print(f"Subjects with trial alerts: {subjects_with_trial_alerts}")

# Test getting alert summary
for subject_id in alerts:
    summary = alert_service.get_threshold_alert_summary(subject_id)
    print(f"\nAlert summary for {subject_id}: {summary}")

# Restore original validation method
alert_service._validate_analyzers = original_validate

Calculating threshold alerts...
Generated alerts for 3 subjects

Subject: sub001
  Feature: total_trials
    above_lower: value=55.0 count=5
  Feature: error_count
    below_upper: value=18.0 count=6

Subject: sub002
  Feature: total_trials
    above_lower: value=44.0 count=1
  Feature: error_count
    below_upper: value=27.0 count=4

Subject: sub003
  Feature: total_trials
    above_lower: value=55.0 count=5
  Feature: error_count
    below_upper: value=18.0 count=6

Subjects with any alert: ['sub001', 'sub002', 'sub003']
Subjects with trial alerts: ['sub001', 'sub002', 'sub003']

Alert summary for sub001: 2 features with alerts: total_trials,error_count (2 total conditions)

Alert summary for sub002: 2 features with alerts: total_trials,error_count (2 total conditions)

Alert summary for sub003: 2 features with alerts: total_trials,error_count (2 total conditions)


In [9]:
# Create sample data for percentile testing
def create_sample_percentile_data():
    """Create sample data for testing percentile-based alerts"""
    # Define strata
    strata = ['Task1_ADVANCED_v3', 'Task1_INTERMEDIATE_v3', 'Task2_BEGINNER_v3']
    
    # Define features with percentile values
    features = {
        'trials': {
            'sub001': {'current': 95.0, 'historical': [80.0, 60.0]},
            'sub002': {'current': 30.0, 'historical': [20.0, 10.0]},
            'sub003': {'current': 50.0, 'historical': [45.0, 55.0]},
            'sub004': {'current': 99.0, 'historical': [98.0, 97.0]},
            'sub005': {'current': 1.5, 'historical': [5.0, 10.0]}
        },
        'errors': {
            'sub001': {'current': 10.0, 'historical': [15.0, 20.0]},
            'sub002': {'current': 85.0, 'historical': [80.0, 75.0]},
            'sub003': {'current': 40.0, 'historical': [45.0, 50.0]},
            'sub004': {'current': 98.5, 'historical': [97.0, 96.0]},
            'sub005': {'current': 1.0, 'historical': [2.0, 3.0]}
        },
        'accuracy': {
            'sub001': {'current': 97.0, 'historical': [95.0, 90.0]},
            'sub002': {'current': 5.0, 'historical': [10.0, 15.0]},
            'sub003': {'current': 60.0, 'historical': [55.0, 50.0]},
            'sub004': {'current': 92.0, 'historical': [90.0, 88.0]},
            'sub005': {'current': 3.0, 'historical': [5.0, 8.0]}
        }
    }
    
    # Create rows for the dataframe
    rows = []
    
    # Current strata - one per subject
    for subject_id in features['trials'].keys():
        # Assign a strata based on subject number (simple rule for testing)
        strata_idx = (int(subject_id[-1]) - 1) % len(strata)
        current_strata = strata[strata_idx]
        
        row = {
            'subject_id': subject_id,
            'strata': current_strata,
            'is_current': True,
            'session_count': 10,
            'first_date': datetime.now() - timedelta(days=30),
            'last_date': datetime.now()
        }
        
        # Add percentile values for each feature
        for feature, subjects in features.items():
            percentile = subjects[subject_id]['current']
            row[f'{feature}_percentile'] = percentile
            row[f'{feature}_processed'] = percentile / 10.0  # Just for testing
        
        rows.append(row)
    
    # Historical strata - vary by subject
    for subject_id in features['trials'].keys():
        # Use different strata for historical data
        for i, hist_strata in enumerate(strata):
            if hist_strata == rows[int(subject_id[-1])-1]['strata']:
                continue  # Skip current strata
                
            # Create historical rows with older dates
            row = {
                'subject_id': subject_id,
                'strata': hist_strata,
                'is_current': False,
                'session_count': 5,
                'first_date': datetime.now() - timedelta(days=90 + i*30),
                'last_date': datetime.now() - timedelta(days=60 + i*30)
            }
            
            # Add percentile values for each feature (use historical if available)
            for feature, subjects in features.items():
                if i < len(subjects[subject_id]['historical']):
                    percentile = subjects[subject_id]['historical'][i]
                else:
                    percentile = subjects[subject_id]['current'] - 10  # Fallback
                    
                row[f'{feature}_percentile'] = percentile
                row[f'{feature}_processed'] = percentile / 10.0  # Just for testing
            
            rows.append(row)
    
    return pd.DataFrame(rows)

print("Creating sample percentile data...")
sample_percentile_data = create_sample_percentile_data()
print(f"Created data with {len(sample_percentile_data)} rows for {len(sample_percentile_data['subject_id'].unique())} subjects")
# Print column names to verify correct structure
print("\nColumns in sample data:")
print(sample_percentile_data.columns.tolist())

# Print a sample row to verify data structure
print("\nSample data row:")
print(sample_percentile_data.iloc[0].to_dict())

Creating sample percentile data...
Created data with 15 rows for 5 subjects

Columns in sample data:
['subject_id', 'strata', 'is_current', 'session_count', 'first_date', 'last_date', 'trials_percentile', 'trials_processed', 'errors_percentile', 'errors_processed', 'accuracy_percentile', 'accuracy_processed']

Sample data row:
{'subject_id': 'sub001', 'strata': 'Task1_ADVANCED_v3', 'is_current': True, 'session_count': 10, 'first_date': Timestamp('2025-02-22 14:29:30.742384'), 'last_date': Timestamp('2025-03-24 14:29:30.742393'), 'trials_percentile': 95.0, 'trials_processed': 9.5, 'errors_percentile': 10.0, 'errors_processed': 1.0, 'accuracy_percentile': 97.0, 'accuracy_processed': 9.7}


In [10]:
# Create a mock QuantileAnalyzer that properly implements the interface
class MockQuantileAnalyzer:
    def __init__(self, sample_data):
        self.sample_data = sample_data
    
    def create_comprehensive_dataframe(self, include_history=False):
        """Return the entire sample dataset if include_history is True, otherwise only current strata"""
        print(f"Mock create_comprehensive_dataframe called with include_history={include_history}")
        if include_history:
            result = self.sample_data.copy()
        else:
            result = self.sample_data[self.sample_data['is_current']].copy()
        print(f"Returning dataframe with {len(result)} rows")
        return result

# Create mock threshold analyzer to satisfy validator
class MockThresholdAnalyzer:
    def __init__(self):
        pass

# Mock the analyzers and attach to utils
utils.quantile_analyzer = MockQuantileAnalyzer(sample_percentile_data)
utils.threshold_analyzer = MockThresholdAnalyzer()

# Override the validation method to make debugging easier
def mock_validate():
    print("Validation check called, returning True")
    return True

alert_service._validate_analyzers = mock_validate


In [11]:
# Try to calculate quantile alerts with debugging
print("\nCalculating quantile alerts...")
try:
    alerts = alert_service.calculate_quantile_alerts()
    if alerts is None:
        print("WARNING: calculate_quantile_alerts returned None")
    else:
        print(f"Generated quantile alerts for {len(alerts)} subjects")
        
        # Display alerts for each subject (current strata only for brevity)
        for subject_id, subject_alerts in alerts.items():
            print(f"\nSubject: {subject_id}")
            print("  Current strata alerts:")
            
            for strata, strata_alerts in subject_alerts['current'].items():
                print(f"    Strata: {strata}")
                
                for feature, alert in strata_alerts.items():
                    print(f"      {feature}: {alert['percentile']:.1f}% - {alert['category']} ({alert['description']})")
except Exception as e:
    print(f"Error calculating alerts: {str(e)}")
    import traceback
    traceback.print_exc()


Calculating quantile alerts...
Validation check called, returning True
Mock create_comprehensive_dataframe called with include_history=True
Returning dataframe with 15 rows
Generated quantile alerts for 5 subjects

Subject: sub001
  Current strata alerts:
    Strata: Task1_ADVANCED_v3
      trials: 95.0% - G (Above Average)
      errors: 10.0% - B (Below Average)
      accuracy: 97.0% - G (Above Average)

Subject: sub002
  Current strata alerts:
    Strata: Task1_INTERMEDIATE_v3
      trials: 30.0% - N (Average)
      errors: 85.0% - N (Average)
      accuracy: 5.0% - B (Below Average)

Subject: sub003
  Current strata alerts:
    Strata: Task2_BEGINNER_v3
      trials: 50.0% - N (Average)
      errors: 40.0% - N (Average)
      accuracy: 60.0% - N (Average)

Subject: sub004
  Current strata alerts:
    Strata: Task1_ADVANCED_v3
      trials: 99.0% - SG (Significantly Above Average)
      errors: 98.5% - SG (Significantly Above Average)
      accuracy: 92.0% - G (Above Average)

Subje

In [12]:
## TESTING COMBINED ALERT ACCESS METHODS

# Set up mock threshold analyzer
print("Setting up mock threshold analyzer...")
threshold_data = pd.DataFrame({
    'subject_id': ['sub001', 'sub002', 'sub003', 'sub001', 'sub002', 'sub003'],
    'session_date': [
        datetime.now() - timedelta(days=5),
        datetime.now() - timedelta(days=5),
        datetime.now() - timedelta(days=5),
        datetime.now() - timedelta(days=1),
        datetime.now() - timedelta(days=1),
        datetime.now() - timedelta(days=1)
    ],
    'total_trials': [90, 70, 100, 85, 65, 95],
    'error_count': [5, 15, 8, 7, 18, 9],
    'total_trials_above_lower': [True, False, True, True, False, True],  # Above 80
    'error_count_below_upper': [True, False, True, True, False, True]    # Below 10
})

threshold_summary = pd.DataFrame({
    'subject_id': ['sub001', 'sub002', 'sub003'],
    'total_sessions': [2, 2, 2],
    'total_trials_above_lower_count': [2, 0, 2],
    'total_trials_above_lower_percent': [100.0, 0.0, 100.0],
    'error_count_below_upper_count': [2, 0, 2],
    'error_count_below_upper_percent': [100.0, 0.0, 100.0]
})

Setting up mock threshold analyzer...


In [13]:
class MockThresholdAnalyzer:
    def __init__(self, data, summary):
        self.data = data
        self.summary = summary
    
    def get_threshold_crossings(self, subject_ids=None, start_date=None, end_date=None):
        result = self.data.copy()
        if subject_ids is not None:
            result = result[result['subject_id'].isin(subject_ids)]
        return result
    
    def get_subject_crossing_summary(self, subject_ids=None):
        result = self.summary.copy()
        if subject_ids is not None:
            result = result[result['subject_id'].isin(subject_ids)]
        return result

# Set up mock quantile analyzer
print("Setting up mock quantile analyzer...")
percentile_data = pd.DataFrame([
    # sub001 - good overall performance
    {
        'subject_id': 'sub001', 'strata': 'Task1_ADVANCED_v3', 'is_current': True,
        'trials_percentile': 92.0, 'errors_percentile': 88.0, 'accuracy_percentile': 95.0,
        'trials_processed': 9.2, 'errors_processed': 8.8, 'accuracy_processed': 9.5,
        'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
    },
    # sub002 - poor overall performance
    {
        'subject_id': 'sub002', 'strata': 'Task1_INTERMEDIATE_v3', 'is_current': True,
        'trials_percentile': 5.0, 'errors_percentile': 2.0, 'accuracy_percentile': 8.0,
        'trials_processed': 0.5, 'errors_processed': 0.2, 'accuracy_processed': 0.8,
        'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
    },
    # sub003 - mixed performance
    {
        'subject_id': 'sub003', 'strata': 'Task1_BEGINNER_v3', 'is_current': True,
        'trials_percentile': 50.0, 'errors_percentile': 75.0, 'accuracy_percentile': 25.0,
        'trials_processed': 5.0, 'errors_processed': 7.5, 'accuracy_processed': 2.5,
        'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
    },
    # sub004 - only in quantile data
    {
        'subject_id': 'sub004', 'strata': 'Task2_ADVANCED_v3', 'is_current': True,
        'trials_percentile': 99.0, 'errors_percentile': 98.0, 'accuracy_percentile': 97.0,
        'trials_processed': 9.9, 'errors_processed': 9.8, 'accuracy_processed': 9.7,
        'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
    }
])

class MockQuantileAnalyzer:
    def __init__(self, data):
        self.data = data
    
    def create_comprehensive_dataframe(self, include_history=False):
        return self.data.copy()

# Attach mock analyzers to utils
utils.threshold_analyzer = MockThresholdAnalyzer(threshold_data, threshold_summary)
utils.quantile_analyzer = MockQuantileAnalyzer(percentile_data)

# Override methods to match our mocks
utils.get_threshold_crossings = utils.threshold_analyzer.get_threshold_crossings
utils.get_subject_threshold_summary = utils.threshold_analyzer.get_subject_crossing_summary

# Override validator to return True
alert_service._validate_analyzers = lambda: True

# Test the combined alert methods
print("\nTesting combined alert access methods...")

Setting up mock quantile analyzer...

Testing combined alert access methods...


In [14]:
# Get combined alerts
combined_alerts = alert_service.get_alerts()
print(f"\nCombined alerts retrieved for {len(combined_alerts)} subjects")

# List which subjects have both types of alerts
subjects_with_both = [
    sid for sid, alerts in combined_alerts.items() 
    if alerts['threshold'] and alerts['quantile']['current']
]
print(f"Subjects with both threshold and quantile alerts: {subjects_with_both}")

# Test subjects with alerts method
print("\nSubjects with different alert criteria:")

all_subjects = alert_service.get_subjects_with_alerts()
print(f"Any alert: {all_subjects}")

threshold_subjects = alert_service.get_subjects_with_alerts(threshold_features=['total_trials'])
print(f"Threshold alerts (total_trials): {threshold_subjects}")

bad_quantile_subjects = alert_service.get_subjects_with_alerts(
    quantile_categories=['SB', 'B']
)
print(f"Bad quantile alerts: {bad_quantile_subjects}")

# Test alert summaries
print("\nAlert summaries:")
for subject_id in combined_alerts:
    summary = alert_service.get_alert_summary(subject_id)
    print(f"{subject_id}: {summary['combined']}")

# Test critical alerts
print("\nCritical alert check:")
for subject_id in combined_alerts:
    has_critical = alert_service.has_critical_alerts(subject_id)
    print(f"{subject_id}: Has critical alerts: {has_critical}")

# Test alert counts
alert_counts = alert_service.get_alert_counts()
print("\nAlert counts across all subjects:")
print(f"Total subjects with alerts: {alert_counts['total_subjects_with_alerts']}")
print(f"Subjects with threshold alerts: {alert_counts['threshold']['subjects_with_alerts']}")
print(f"Subjects with quantile alerts: {alert_counts['quantile']['subjects_with_alerts']}")
print(f"Subjects with critical alerts: {alert_counts['subjects_with_critical_alerts']}")
print(f"Quantile category distribution: {alert_counts['quantile']['category_counts']}")


Combined alerts retrieved for 4 subjects
Subjects with both threshold and quantile alerts: ['sub002', 'sub001', 'sub003']

Subjects with different alert criteria:
Any alert: ['sub002', 'sub001', 'sub003', 'sub004']
Threshold alerts (total_trials): ['sub002', 'sub001', 'sub003']
Bad quantile alerts: ['sub002']

Alert summaries:
sub002: Threshold: 2 features with alerts: total_trials,error_count (2 total conditions) | Quantile: 1 Significantly Below Average: errors | 2 Below Average: trials, accuracy | in strata: Task1_INTERMEDIATE_v3
sub001: Threshold: 2 features with alerts: total_trials,error_count (2 total conditions) | Quantile: 3 Above Average: trials, errors, accuracy | in strata: Task1_ADVANCED_v3
sub003: Threshold: 2 features with alerts: total_trials,error_count (2 total conditions) | Quantile: 3 Average features | in strata: Task1_BEGINNER_v3
sub004: Threshold: No threshold_alerts | Quantile: 1 Above Average: accuracy | 2 Significantly Above Average: trials, errors | in strat

In [15]:
## TESTING APP UTILS INTEGRATION

# Function to set up mock data for testing
def setup_mock_data():
    # Create sample session data
    session_data = pd.DataFrame({
        'subject_id': ['sub001', 'sub002', 'sub003'] * 2,
        'session_date': [
            datetime.now() - timedelta(days=5),
            datetime.now() - timedelta(days=5),
            datetime.now() - timedelta(days=5),
            datetime.now() - timedelta(days=1),
            datetime.now() - timedelta(days=1),
            datetime.now() - timedelta(days=1)
        ],
        'total_trials': [90, 70, 100, 85, 65, 95],
        'error_count': [5, 15, 8, 7, 18, 9],
        'session': ['s1', 's1', 's1', 's2', 's2', 's2']
    })
    
    # Create mock threshold data with crossing columns
    threshold_data = session_data.copy()
    threshold_data['total_trials_above_lower'] = threshold_data['total_trials'] >= 80
    threshold_data['error_count_below_upper'] = threshold_data['error_count'] <= 10
    
    # Create mock percentile data
    percentile_data = pd.DataFrame([
        # sub001 - good performance
        {
            'subject_id': 'sub001', 'strata': 'Task1_ADVANCED_v3', 'is_current': True,
            'trials_percentile': 92.0, 'errors_percentile': 88.0, 'accuracy_percentile': 95.0,
            'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
        },
        # sub002 - poor performance
        {
            'subject_id': 'sub002', 'strata': 'Task1_INTERMEDIATE_v3', 'is_current': True,
            'trials_percentile': 5.0, 'errors_percentile': 2.0, 'accuracy_percentile': 8.0,
            'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
        },
        # sub003 - mixed performance
        {
            'subject_id': 'sub003', 'strata': 'Task1_BEGINNER_v3', 'is_current': True,
            'trials_percentile': 50.0, 'errors_percentile': 75.0, 'accuracy_percentile': 25.0,
            'first_date': datetime.now() - timedelta(days=30), 'last_date': datetime.now()
        }
    ])
    
    return session_data, threshold_data, percentile_data

# Create a fresh AppUtils instance
utils = AppUtils()

# Set up mock data
session_data, threshold_data, percentile_data = setup_mock_data()

In [16]:
# Create mock classes
class MockDataLoader:
    def __init__(self, data):
        self.data = data
    
    def get_data(self):
        return self.data

class MockThresholdAnalyzer:
    def __init__(self, data):
        self.data = data
    
    # Use this method name for the test to match app_utils.py
    def get_threshold_crossings(self, subject_ids=None, start_date=None, end_date=None):
        result = self.data.copy()
        if subject_ids is not None:
            result = result[result['subject_id'].isin(subject_ids)]
        return result
    
    def get_subject_crossing_summary(self, subject_ids=None):
        # Create a simple summary from the data
        summary_data = []
        for subject_id in self.data['subject_id'].unique():
            if subject_ids is not None and subject_id not in subject_ids:
                continue
                
            subject_df = self.data[self.data['subject_id'] == subject_id]
            
            # Count how many sessions passed each threshold
            trials_passed = subject_df['total_trials_above_lower'].sum()
            errors_passed = subject_df['error_count_below_upper'].sum()
            
            summary_data.append({
                'subject_id': subject_id,
                'total_sessions': len(subject_df),
                'total_trials_above_lower_count': trials_passed,
                'total_trials_above_lower_percent': (trials_passed / len(subject_df)) * 100,
                'error_count_below_upper_count': errors_passed,
                'error_count_below_upper_percent': (errors_passed / len(subject_df)) * 100
            })
            
        return pd.DataFrame(summary_data)

class MockQuantileAnalyzer:
    def __init__(self, data):
        self.data = data
    
    def create_comprehensive_dataframe(self, include_history=False):
        return self.data.copy()

# Set up the AppUtils instance with our mocks
utils.data_loader = MockDataLoader(session_data)
utils.threshold_analyzer = MockThresholdAnalyzer(threshold_data)
utils.quantile_analyzer = MockQuantileAnalyzer(percentile_data)

# Add mock methods that match EXACTLY what AlertService expects
# Note the singular "crossing" to match your app_utils.py method
utils.get_threshold_crossing = utils.threshold_analyzer.get_threshold_crossings
utils.get_subject_threshold_summary = utils.threshold_analyzer.get_subject_crossing_summary

In [17]:
# Test the integrated alert service
print("Testing AppUtils integration with AlertService...")

# Initialize the alert service through AppUtils
alert_service = utils.initialize_alert_service()
print(f"Alert service initialized: {alert_service is not None}")

# Get alerts through AppUtils
alerts = utils.get_alerts()
print(f"\nGot alerts for {len(alerts)} subjects through AppUtils")

# Check if subjects have alerts
for subject_id in alerts:
    has_critical = utils.has_critical_alerts(subject_id)
    summary = utils.get_alert_summary(subject_id)
    print(f"\nSubject {subject_id}:")
    print(f"  Has critical alerts: {has_critical}")
    print(f"  Alert summary: {summary['combined']}")

# Get subjects with specific alert criteria
bad_performers = utils.get_subjects_with_alerts(
    quantile_categories=['SB', 'B']
)
print(f"\nSubjects with bad performance: {bad_performers}")

# Get alert counts
alert_counts = utils.get_alert_counts()
print("\nAlert counts:")
print(f"Total subjects with alerts: {alert_counts['total_subjects_with_alerts']}")
print(f"Subjects with critical alerts: {alert_counts['subjects_with_critical_alerts']}")

Testing AppUtils integration with AlertService...
Alert service initialized: True

Got alerts for 3 subjects through AppUtils

Subject sub003:
  Has critical alerts: True
  Alert summary: Threshold: 2 features with alerts: total_trials,error_count (2 total conditions) | Quantile: 3 Average features | in strata: Task1_BEGINNER_v3

Subject sub001:
  Has critical alerts: True
  Alert summary: Threshold: 2 features with alerts: total_trials,error_count (2 total conditions) | Quantile: 3 Above Average: trials, errors, accuracy | in strata: Task1_ADVANCED_v3

Subject sub002:
  Has critical alerts: True
  Alert summary: Threshold: No threshold_alerts | Quantile: 1 Significantly Below Average: errors | 2 Below Average: trials, accuracy | in strata: Task1_INTERMEDIATE_v3

Subjects with bad performance: ['sub002']

Alert counts:
Total subjects with alerts: 3
Subjects with critical alerts: 3
